In [1]:
# Install all required packages in the current environment
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install "ultralytics" "numpy<2" "torch" "torchvision" "Pillow"

In [2]:
!{sys.executable} -m pip install "ultralytics numpy<2"
import numpy
print("NumPy version:", numpy.__version__)

ERROR: Invalid requirement: 'ultralytics numpy<2': Expected end or semicolon (after name and no valid version specifier)
    ultralytics numpy<2
                ^
NumPy version: 1.26.4


In [3]:
import numpy
print("NumPy version:", numpy.__version__)

import torch
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights
from ultralytics import YOLO
from PIL import Image
import torchvision.transforms as transforms
from pathlib import Path

NumPy version: 1.26.4


In [4]:
# --- Build your ResNet wrapper ---
OUT_SIZE = 10  # adjust to your dataset
resnet = resnet50(weights=None)  # or ResNet50_Weights.IMAGENET1K_V2
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, OUT_SIZE)

In [5]:
class ResNet(nn.Module):
    def __init__(self, model):
        super(ResNet, self).__init__()
        self.model = model

    def forward(self, x):
        return self.model(x)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = ResNet(resnet).to(device)
model.eval()

ResNet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0)

In [6]:
# --- Load YOLO ---
yolo_model = YOLO("yolo11n.pt")

In [7]:
# --- Transform for ResNet ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

In [8]:
# --- Define directories and class mapping ---
data_dir = Path("/Users/keitasakurai/PycharmProjects/CDB/blood_cell_images")
class_names = {
    "seg_neutrophil": 0,
    "erythroblast": 1,
    "myeloblast": 2,
    "monocyte": 3,
    "basophil": 4
}

# --- Iterate over all images in all subdirectories ---
for cell_type, class_idx in class_names.items():
    image_paths = data_dir.glob(f"{cell_type}/*.jpg")  # adjust extension if needed
    for image_path in image_paths:
        # Run YOLO detection
        results = yolo_model.predict(str(image_path), verbose=False)

        for r in results:
            img = Image.open(r.path).convert("RGB")
            boxes = r.boxes.xyxy  # bounding boxes

            preds = []
            for box in boxes:
                x1, y1, x2, y2 = map(int, box)
                crop = img.crop((x1, y1, x2, y2))

                # Preprocess and classify with ResNet
                input_tensor = transform(crop).unsqueeze(0).to(device)
                with torch.no_grad():
                    outputs = model(input_tensor)
                    _, pred = outputs.max(1)
                    preds.append(pred.item())

            print(f"{image_path.name} predictions: {preds}")

NGS_1133.jpg predictions: [0, 0, 0, 0, 0, 0, 0]
NGS_1655.jpg predictions: [0, 0]
NGS_1641.jpg predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
NGS_2348.jpg predictions: [0, 0, 0, 0, 0]
NGS_5341.jpg predictions: [0, 0, 0, 0, 0, 0, 0, 0]
NGS_1669.jpg predictions: [0, 0, 0]
NGS_6112.jpg predictions: []
NGS_1682.jpg predictions: [0, 0, 0]
NGS_2638.jpg predictions: [0, 0, 0]
NGS_5143.jpg predictions: [0, 0, 0, 0]
NGS_2162.jpg predictions: [0, 0]
NGS_2610.jpg predictions: []
NGS_4261.jpg predictions: [0, 0, 0]
NGS_0952.jpg predictions: [0]
NGS_6489.jpg predictions: [0, 0, 0]
NGS_0946.jpg predictions: [0, 0, 0, 0]
NGS_1480.jpg predictions: [0, 0]
NGS_1523.jpg predictions: [0]
NGS_2994.jpg predictions: [0]
NGS_3452.jpg predictions: [0]
NGS_1251.jpg predictions: [0, 0]
NGS_7620.jpg predictions: []
NGS_5989.jpg predictions: [0]
NGS_1537.jpg predictions: [0, 0, 0, 0, 0]
NGS_2957.jpg predictions: [0, 0, 0, 0, 0, 0]
NGS_1292.jpg predictions: [0]
NGS_6728.jpg predictions: [0, 0]
NGS_1053.jpg predictions:

NGS_7234.jpg predictions: [0, 0, 0]
NGS_8113.jpg predictions: []
NGS_3046.jpg predictions: [0, 0]
NGS_3720.jpg predictions: [0, 0, 0, 0]
NGS_1137.jpg predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0]
NGS_6670.jpg predictions: [0, 0, 0, 0, 0]
NGS_6116.jpg predictions: [0, 0, 0]
NGS_5450.jpg predictions: [0]
NGS_4996.jpg predictions: []
NGS_7253.jpg predictions: [0, 0, 0]
NGS_6165.jpg predictions: [0, 0, 0, 0, 0, 0]
NGS_6603.jpg predictions: []
NGS_6617.jpg predictions: [0, 0, 0, 0]
NGS_7284.jpg predictions: [0, 0, 0, 0]
NGS_1352.jpg predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
NGS_1420.jpg predictions: []
NGS_5120.jpg predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0]
NGS_0058.jpg predictions: [0, 0, 0, 0]
NGS_8438.jpg predictions: [0]
NGS_0064.jpg predictions: [0, 0]
NGS_2698.jpg predictions: [0]
NGS_1391.jpg predictions: [0]
NGS_5691.jpg predictions: [0]
NGS_7904.jpg predictions: [0]
NGS_5875.jpg predictions: [0, 0, 0, 0, 0]
NGS_7910.jpg predictions: [0, 0, 0, 0]
NGS_1226.jpg predictions: [0, 0, 0]
NGS_3

NGS_7451.jpg predictions: [0]
NGS_3637.jpg predictions: [0, 0, 0]
NGS_2267.jpg predictions: [0]
NGS_6773.jpg predictions: []
NGS_5093.jpg predictions: [0, 0, 0, 0]
NGS_3384.jpg predictions: [0, 0, 0, 0, 0, 0]
NGS_3390.jpg predictions: [0, 0, 0]
NGS_5087.jpg predictions: [0, 0, 0, 0]
NGS_7860.jpg predictions: [0, 0, 0, 0, 0, 0, 0]
NGS_1550.jpg predictions: [0, 0]
NGS_1578.jpg predictions: [0, 0, 0, 0, 0, 0, 0]
NGS_3596.jpg predictions: [0, 0]
NGS_0921.jpg predictions: [0, 0, 0, 0, 0, 0, 0]
NGS_5681.jpg predictions: [0, 0, 0, 0]
NGS_5695.jpg predictions: []
NGS_5871.jpg predictions: [0, 0, 0, 0]
NGS_3555.jpg predictions: [0, 0]
NGS_1356.jpg predictions: [0]
NGS_7727.jpg predictions: [0, 0]
NGS_2677.jpg predictions: [0]
NGS_3970.jpg predictions: [0]
NGS_3964.jpg predictions: [0, 0, 0, 0, 0]
NGS_7531.jpg predictions: [0, 0, 0, 0]
NGS_8158.jpg predictions: [0]
NGS_7519.jpg predictions: [0]
NGS_3766.jpg predictions: [0, 0, 0]
NGS_4035.jpg predictions: [0, 0, 0]
NGS_6805.jpg predictions: [0, 

NGS_0929.jpg predictions: [0, 0, 0, 0, 0, 0, 0]
NGS_0083.jpg predictions: [0, 0, 0, 0]
NGS_2864.jpg predictions: [0, 0, 0]
NGS_7897.jpg predictions: []
NGS_0120.jpg predictions: []
NGS_1216.jpg predictions: [0, 0, 0, 0]
NGS_7101.jpg predictions: []
NGS_7840.jpg predictions: [0, 0, 0, 0, 0, 0]
NGS_4391.jpg predictions: [0]
NGS_0861.jpg predictions: [0, 0, 0]
NGS_2910.jpg predictions: [0]
NGS_0875.jpg predictions: [0, 0, 0, 0, 0]
NGS_0322.jpg predictions: [0, 0, 0]
NGS_6753.jpg predictions: [0, 0, 0, 0, 0]
NGS_8024.jpg predictions: []
NGS_5299.jpg predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
NGS_6974.jpg predictions: [0]
NGS_4192.jpg predictions: [0, 0, 0]
NGS_0486.jpg predictions: [0, 0, 0, 0]
NGS_5515.jpg predictions: []
NGS_7470.jpg predictions: [0, 0, 0, 0, 0]
NGS_1767.jpg predictions: [0, 0, 0, 0, 0, 0, 0, 0]
NGS_0874.jpg predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
NGS_5918.jpg predictions: [0, 0]
NGS_6587.jpg predictions: [0, 0, 0, 0]
NGS_0109.jpg predictions: [0, 0, 0]
NGS_6578.jpg

NGS_5944.jpg predictions: [0, 0, 0]
NGS_3312.jpg predictions: [0, 0, 0, 0]
NGS_7606.jpg predictions: [0]
NGS_3306.jpg predictions: []
NGS_6256.jpg predictions: [0, 0, 0, 0]
NGS_6530.jpg predictions: [0]
NGS_0948.jpg predictions: [0, 0, 0, 0, 0, 0, 0, 0]
NGS_4535.jpg predictions: [0, 0, 0, 0, 0, 0, 0, 0]
NGS_7996.jpg predictions: []
NGS_6685.jpg predictions: []
NGS_0586.jpg predictions: [0, 0, 0, 0, 0]
NGS_6691.jpg predictions: []
NGS_1115.jpg predictions: [0, 0, 0, 0]
NGS_5415.jpg predictions: [0, 0, 0]
NGS_1129.jpg predictions: [0]
NGS_6134.jpg predictions: [0]
NGS_0223.jpg predictions: [0, 0, 0, 0, 0]
NGS_6652.jpg predictions: [0, 0, 0, 0]
NGS_4053.jpg predictions: [0]
NGS_5417.jpg predictions: []
NGS_0989.jpg predictions: [0]
NGS_7770.jpg predictions: [0]
NGS_7957.jpg predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0]
NGS_2807.jpg predictions: []
NGS_0625.jpg predictions: [0]
NGS_6254.jpg predictions: []
NGS_0619.jpg predictions: [0, 0, 0, 0]
NGS_8251.jpg predictions: [0]
NGS_4480.jpg predict

...

MO_356393.jpg predictions: [0, 0, 0, 0]
MO_266522.jpg predictions: [0, 0, 0, 0, 0]
MO_145357.jpg predictions: [0]
MO_108531.jpg predictions: [0]
MO_775971.jpg predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
MO_61107.jpg predictions: [0]
MO_773253.jpg predictions: [0, 0]
MO_406112.jpg predictions: [0]
MO_987913.jpg predictions: [0, 0, 0, 0]
MO_676317.jpg predictions: [0, 0, 0, 0, 0, 0, 0, 0]
MO_382707.jpg predictions: [0]
MO_402348.jpg predictions: [0]
MO_461638.jpg predictions: [0, 0]
MO_51985.jpg predictions: [0]
MO_66318.jpg predictions: [0, 0, 0, 0, 0, 0]
MO_383141.jpg predictions: [0, 0, 0, 0]
MO_74720.jpg predictions: [0, 0, 0, 0, 0, 0, 0]
MO_340650.jpg predictions: []
MO_972971.jpg predictions: [0]
MO_420300.jpg predictions: [0, 0, 0]
MO_717500.jpg predictions: [0, 0]
MO_646983.jpg predictions: [0]
MO_661482.jpg predictions: [0, 0, 0, 0]
MO_122086.jpg predictions: [0, 0, 0]
MO_96000.jpg predictions: [0]
MO_338729.jpg predictions: [0, 0, 0]
MO_896406.jpg predictions: [0, 0, 0, 0, 0

BA_221009.jpg predictions: [0]
BA_183362.jpg predictions: [0, 0, 0, 0, 0, 0, 0]
BA_880798.jpg predictions: [0]
BA_636027.jpg predictions: [0, 0, 0]
BA_237864.jpg predictions: [0, 0, 0, 0]
BA_490880.jpg predictions: [0]
BA_852185.jpg predictions: [0, 0]
BA_995394.jpg predictions: [0, 0, 0, 0, 0, 0, 0, 0]
BA_540646.jpg predictions: [0, 0, 0, 0]
BA_794444.jpg predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
BA_178853.jpg predictions: [0]
BA_344479.jpg predictions: [0]
BA_332289.jpg predictions: [0]
BA_40355.jpg predictions: [0, 0, 0, 0, 0, 0, 0]
BA_421910.jpg predictions: [0, 0, 0, 0]
BA_684441.jpg predictions: [0, 0, 0]
BA_999747.jpg predictions: [0]
BA_78609.jpg predictions: [0]
BA_929127.jpg predictions: [0, 0]
BA_427583.jpg predictions: [0, 0, 0]
BA_736344.jpg predictions: [0, 0]
BA_518773.jpg predictions: [0, 0]
BA_296311.jpg predictions: [0]
BA_351114.jpg predictions: [0, 0]
BA_374431.jpg predictions: [0, 0]
BA_407690.jpg predictions: [0, 0, 0]
BA_760857.jpg predictions: [0, 0]
BA_759768

BA_594501.jpg predictions: [0, 0]
BA_855924.jpg predictions: [0]
BA_887335.jpg predictions: [0, 0, 0, 0]
BA_864938.jpg predictions: [0, 0, 0]
BA_618018.jpg predictions: [0, 0, 0, 0, 0, 0, 0]
BA_895902.jpg predictions: [0, 0]
BA_131174.jpg predictions: [0]
BA_18821.jpg predictions: [0]
BA_476625.jpg predictions: [0, 0, 0, 0, 0]
BA_913408.jpg predictions: [0, 0]
BA_779592.jpg predictions: [0]
BA_42246.jpg predictions: [0]
BA_345077.jpg predictions: [0]
BA_249665.jpg predictions: [0]
BA_93147.jpg predictions: [0, 0, 0, 0, 0]
BA_925384.jpg predictions: [0, 0, 0]
BA_915247.jpg predictions: [0]
BA_795062.jpg predictions: [0, 0]
BA_112418.jpg predictions: [0]
BA_721289.jpg predictions: [0]
BA_472669.jpg predictions: [0, 0]
BA_27810.jpg predictions: [0]
BA_728520.jpg predictions: [0]
BA_509224.jpg predictions: [0]
BA_127306.jpg predictions: [0, 0]
BA_581736.jpg predictions: [0]
BA_58746.jpg predictions: [0, 0, 0]
BA_558057.jpg predictions: [0, 0]
BA_453670.jpg predictions: [0]
BA_584667.jpg pr

BA_615294.jpg predictions: [0, 0, 0]
BA_126411.jpg predictions: [0, 0, 0]
BA_396430.jpg predictions: [0]
BA_625394.jpg predictions: [0, 0, 0, 0]
BA_791143.jpg predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0]
BA_759075.jpg predictions: [0, 0]
BA_201200.jpg predictions: [0]
BA_139733.jpg predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
BA_361292.jpg predictions: [0, 0]
BA_179459.jpg predictions: [0, 0, 0, 0, 0]
BA_855901.jpg predictions: [0, 0]
BA_584910.jpg predictions: [0]
BA_545194.jpg predictions: [0]
BA_425674.jpg predictions: [0, 0, 0, 0, 0]
BA_844617.jpg predictions: [0, 0]
BA_417832.jpg predictions: [0, 0, 0, 0]
BA_200697.jpg predictions: [0]
BA_301562.jpg predictions: [0]
BA_22197.jpg predictions: [0]
BA_413473.jpg predictions: [0, 0]
BA_699269.jpg predictions: [0, 0, 0, 0]
BA_920815.jpg predictions: [0, 0, 0, 0, 0, 0, 0]
BA_708202.jpg predictions: [0]
BA_635801.jpg predictions: [0, 0]
BA_785668.jpg predictions: [0]
BA_32179.jpg predictions: [0]
BA_283950.jpg predictions: [0]
BA_383

BA_583166.jpg predictions: [0]
BA_473295.jpg predictions: [0, 0, 0, 0]
BA_290341.jpg predictions: [0]
BA_174731.jpg predictions: [0, 0]
BA_315376.jpg predictions: [0, 0, 0, 0]
BA_241069.jpg predictions: [0]
BA_545901.jpg predictions: [0, 0, 0, 0, 0, 0, 0, 0]
BA_277242.jpg predictions: [0]
BA_248930.jpg predictions: [0, 0, 0, 0, 0]
BA_900142.jpg predictions: [0, 0, 0, 0, 0]
BA_152708.jpg predictions: [0]
BA_564059.jpg predictions: [0, 0]
BA_598745.jpg predictions: [0, 0, 0, 0, 0, 0, 0]
BA_198565.jpg predictions: [0]
BA_819177.jpg predictions: []
BA_926393.jpg predictions: [0, 0, 0, 0]
BA_229935.jpg predictions: [0, 0]
BA_46660.jpg predictions: [0]
BA_139552.jpg predictions: [0, 0, 0, 0, 0, 0]
BA_121453.jpg predictions: [0]
BA_403405.jpg predictions: [0, 0, 0]
BA_55185.jpg predictions: [0, 0]
BA_265929.jpg predictions: [0]
BA_461900.jpg predictions: [0, 0, 0]
BA_166254.jpg predictions: [0, 0, 0, 0]
BA_991920.jpg predictions: [0, 0, 0, 0, 0, 0, 0]
BA_666027.jpg predictions: [0, 0, 0]
BA_5

In [9]:
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

all_preds = []
all_labels = []

# --- Iterate over all images ---
for cell_type, class_idx in class_names.items():
    image_paths = data_dir.glob(f"{cell_type}/*.jpg")
    for image_path in image_paths:
        results = yolo_model.predict(str(image_path), verbose=False)

        for r in results:
            img = Image.open(r.path).convert("RGB")
            boxes = r.boxes.xyxy  # bounding boxes

            for box in boxes:
                x1, y1, x2, y2 = map(int, box)
                crop = img.crop((x1, y1, x2, y2))

                # Preprocess and classify with ResNet
                input_tensor = transform(crop).unsqueeze(0).to(device)
                with torch.no_grad():
                    outputs = model(input_tensor)
                    _, pred = outputs.max(1)

                all_preds.append(pred.item())
                all_labels.append(class_idx)

# --- Compute Accuracy ---
acc = accuracy_score(all_labels, all_preds)
print(f"Overall Accuracy: {acc:.4f}")

# --- Confusion Matrix ---
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names.keys(),
            yticklabels=class_names.keys())
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

KeyboardInterrupt: 